In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [2]:
from langchain.output_parsers import PydanticOutputParser
from typing import List
from pydantic import BaseModel, Field, field_validator
class Suggestion(BaseModel):
    words: List[str] = Field(
        description="A list of suggestions to substitute the target word in the context.")
    
    reasons: List[str] = Field(
        description="A list of reasons for the suggestions.")
py_parser= PydanticOutputParser(pydantic_object=Suggestion)

In [3]:
misformatted_input = {"words":["conduct","manner"],"reasoning":["refers to the way someone acts in a particular situation.","1ChainPrompt.py"]}
py_parser.parse(misformatted_input)

ValidationError: 1 validation error for Generation
text
  str type expected (type=type_error.str)

In [7]:
from langchain.output_parsers import OutputFixingParser
op=OutputFixingParser.from_llm(llm=llm,parser=py_parser)
op.parse(misformatted_input)

ValidationError: 1 validation error for Generation
text
  str type expected (type=type_error.str)

In [5]:
from langchain.output_parsers import RetryOutputParser
from langchain.prompts import PromptTemplate
template = """
Offer a list of suggestions to substitue the specified target_word based the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": py_parser.get_format_instructions()}
)
pf=prompt.format_prompt(target_word="conduct",context="refers to the way someone acts in a particular situation.")
rp=RetryOutputParser.from_llm(llm=llm,parser=py_parser)
rp.parse_with_prompt(misformatted_input,pf)

ValidationError: 1 validation error for Generation
text
  str type expected (type=type_error.str)

In [6]:
from langchain.output_parsers import RetryOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI  # Example LLM

# Initialize the language model (make sure to replace with your model)
llm = ChatOpenAI(model="gpt-3.5-turbo")

# Ensure py_parser is defined and initialized
# Example: py_parser = SomeOutputParser()  # Replace with actual parser initialization

template = """
Offer a list of suggestions to substitute the specified target_word based on the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""

# Create the prompt template
prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": py_parser.get_format_instructions()}
)

# Format the prompt
pf = prompt.format_prompt(target_word="conduct", context="refers to the way someone acts in a particular situation.")

# Initialize the RetryOutputParser
rp = RetryOutputParser.from_llm(llm=llm, parser=py_parser)

# Ensure misformatted_input is defined (this should be the output from the LLM)
misformatted_input = {"words":["conduct","manner"],"reasoning":["refers to the way someone acts in a particular situation.","1ChainPrompt.py"]}

# Parse the output with the prompt
try:
    result = rp.parse_with_prompt(misformatted_input, pf)
    print("Parsed Result:", result)
except Exception as e:
    print("Error during parsing:", e)

Error during parsing: 1 validation error for Generation
text
  str type expected (type=type_error.str)
